In [1]:
from generate_patient_dataset import GeneratePatientDataset
data = GeneratePatientDataset("207", '../data/')

readings = data.readings
annotations = data.annotations
mapping = data.filtered_mapping

In [2]:
sequences_mapping = [list(pair) for pair in mapping.keys()]
labels = [annotation.annotation for annotation in mapping.values()]

In [7]:
sequences = []
for sequence in sequences_mapping:
    entries = []
    for entry in sequence:
        entries.append(entry)
    sequences.append(entries)

In [8]:
# Get max. length for Padding
max_length = 0

for key, val in mapping.items():
    current_length = len(key)
    if current_length > max_length:
        max_length = current_length

print("Maximum sequence length:", max_length)

Maximum sequence length: 924


In [6]:
import pandas as pd

# Load the annotations data
annotations_path = '/mnt/data/207annotations.csv'
annotations = pd.read_csv(annotations_path)

# Convert the time column to minutes
annotations['Time_min'] = annotations['Time'] / 60

# Define the intervals for analysis
intervals = [(0, 7.5), (7.5, 15), (15, 22.5), (22.5, 30)]

# Focus on the specified classes
classes_of_interest = ['L', '!', 'A', 'V', 'E']

# Function to count occurrences of classes in each interval
def count_classes_in_interval(df, interval, classes):
    start, end = interval
    interval_data = df[(df['Time_min'] >= start) & (df['Time_min'] < end)]
    return interval_data['Type'].value_counts().reindex(classes, fill_value=0)

# Count occurrences for each interval
class_counts = {f'{start}-{end} min': count_classes_in_interval(annotations, (start, end), classes_of_interest)
                for start, end in intervals}

class_counts_df = pd.DataFrame(class_counts)
class_counts_df


FileNotFoundError: [Errno 2] No such file or directory: '../data/207annotations.csv'

In [9]:
print(sequences[0])

[(-5.211844051217207, -18.51995806079368), (-5.2357241206986105, -18.616326784629837), (-5.285698223776592, -18.693978674047614), (-5.357523277906973, -18.712687951757914), (-5.397242765375056, -18.611867986408207), (-5.307858703048657, -18.349240118466913), (-4.991022159004928, -17.968579957436003), (-4.4144719244539425, -17.65694015461231), (-3.6616361574511815, -17.721887437694456), (-2.8985522558024033, -18.428712109275278), (-2.238971473034444, -19.745485279570268), (-1.6043515064110938, -21.21446253853546), (-0.7416500155162371, -22.180193248607488), (0.5568109286987171, -22.29561659240709), (2.2156007126358896, -21.85741972137575), (3.868392303188088, -21.587910913364194), (5.105050965450578, -22.015475968203113), (5.736952277989448, -23.02218932430152), (5.846623459200883, -23.970962275973342), (5.626554774469879, -24.28154377320464), (5.209314503573912, -23.919943803466033), (4.651757137082287, -23.364579188942443), (4.048296200047004, -23.114020615979584), (3.6254751575887894

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pre-Processing
def preprocess_sequence(sequences, labels, max_length):
    # Convert labels to integer indices
    label_to_index = {'L': 0, 'A': 1, 'V': 2, '!': 3, 'E': 4}
    indices = [label_to_index[label] for label in labels]

    # Padding
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post', value=0)
    
    return padded_sequences, indices


In [12]:
preprocessed_sequences, indices = preprocess_sequence(sequences, labels, max_length)

In [15]:
print(preprocessed_sequences[5])
print(indices)

[[-168   27]
 [-163   28]
 [-155   32]
 ...
 [   0    0]
 [   0    0]
 [   0    0]]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 2, 0, 0, 0, 2, 2, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [83]:
from sklearn.model_selection import train_test_split

# Split data into train/test
train_x, test_x, train_y, test_y = train_test_split(
    sequences, labels, test_size=0.5, random_state=42, stratify=labels
)

In [84]:
import numpy as np

# Check for data balance between train test
train_class_distribution = np.bincount(train_y)
test_class_distribution = np.bincount(test_y)

print("Training Set Class Distribution:", train_class_distribution)
print("Testing Set Class Distribution:", test_class_distribution)

ValueError: invalid literal for int() with base 10: 'L'